In [4]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

groq_api_key=os.getenv("GROQ_API_KEY")

llm= ChatGroq(groq_api_key=groq_api_key, model="Llama3-8b-8192")

llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7b9422e9a5d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7b9422e9afd0>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


In [11]:
!pip install langchain_huggingface  

  Using cached langchain_huggingface-0.3.0-py3-none-any.whl.metadata (996 bytes)
Using cached langchain_huggingface-0.3.0-py3-none-any.whl (27 kB)


In [6]:
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [7]:
import bs4
loader=WebBaseLoader(
    web_path=("https://en.wikipedia.org/wiki/Artificial_intelligence",)
    # bs_kwargs=dict(
    #     parse_only=bs4.SoupStrainer(
    #         class_=("post-content","post-title","post-header")
    #     )
    # ),
)
doc=loader.load()

In [19]:
text_splitter = RecursiveCharacterTextSplitter(    chunk_size=1000,chunk_overlap=200)
splits=text_splitter.split_documents(doc)
db = Chroma.from_documents(
    documents=splits,
    embedding=embeddings
)

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7b933c135f90>, search_kwargs={})

In [12]:
system_prompt=(
    "You are an assistant for answering questions"
    "Use the following pieces of context to answer the question"
    "{context}"
    "If you don't know the answer, just say that you don't know."
    "answer concise"
)

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),

    ]
)

In [13]:
question_answer_chain=create_stuff_documents_chain(llm, prompt)

rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [14]:
response=rag_chain.invoke({"input":"What is Artificial Intelligence?"})
response['answer']

'Artificial Intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making.'

In [17]:
##adding chat history

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt=(
    "Given a chat history and a question"
    "which might reference context in the chat history"
    "Formulate a standalone question"
    "that can be understood"
    "without the chat history do not answer the question"
    "just formulate it if needed and otherwise return it as it."
)

contextualize_q_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}"),
    ]
)


In [20]:
history_aware_retriever=create_history_aware_retriever(llm,retriever,contextualize_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7b933c135f90>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChu

In [24]:
qa_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}"),
    ]
)

In [25]:
question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [29]:
from langchain_core.messages import AIMessage, HumanMessage
chat_history=[]
question="What are traditional goals of AI?"
response1=rag_chain.invoke({"input":question,"chat_history":chat_history})

chat_history.extend(
    [HumanMessage(content=question), AIMessage(content=response1['answer'])]
)
# response1['answer']

question1="WHat they did to achieve these goals?"
response2=rag_chain.invoke({"input":question1,"chat_history":chat_history}) 
response2['answer']

'According to the text, AI researchers have adapted and integrated a wide range of techniques to achieve these goals, including:\n\n1. Search and mathematical optimization\n2. Formal logic\n3. Artificial neural networks\n4. Methods based on statistics, operations research, and economics\n5. Drawing upon psychology, linguistics, philosophy, neuroscience, and other fields'